In [143]:
import os
import librosa, librosa.display
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
import IPython.display as ipd
import numpy as np
from scipy.fftpack import fft
import shutil
from pydub import AudioSegment

from functools import partial
#import ftransc

In [144]:
def movedir(oripath,label):
    replacestr = "\\test\\"+label+"\\"
    new_path = oripath.replace(label,replacestr)
    return  new_path
def load_audio(file_path):
    waveform, sample_rate = librosa.load(file_path)
    return waveform, sample_rate

def load_audio_files(path: str, label:str):
    train_dataset = []
    test_dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))
    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)
        path_sp = path.split("\\")
        label = path_sp[-1]
        new_path = movedir(path,label)     
        if i%5 != 0:
        # Load audio
            waveform, sample_rate = load_audio(file_path)
            audio_segment = AudioSegment.from_file(file_path)
            train_dataset.append([waveform, sample_rate,audio_segment])
        else :
            waveform, sample_rate = load_audio(file_path)
            audio_segment = AudioSegment.from_file(file_path)
            test_dataset.append([waveform, sample_rate,audio_segment])
    return train_dataset,test_dataset



In [145]:
#dustan_baby
burping_train_data,burping_test_data= load_audio_files('./data_eval/sound_data/dustan_baby/burpme/', 'burpme')
tired_train_data,tired_test_data = load_audio_files('./data_eval/sound_data/dustan_baby/tired_owh/', 'tired')
hungry_train_data,hungry_test_data = load_audio_files('./data_eval/sound_data/dustan_baby/hungry_Nah/', 'hungry')
poop_train_data,poop_test_data = load_audio_files('./data_eval/sound_data/dustan_baby/poop_eairh/', 'poop')
discomfort_train_data,discomfort_test_data = load_audio_files('./data_eval/sound_data/dustan_baby/uncomfortable_heh/', 'discomfort')

In [146]:
def mel_spectrogram(waveform,sample,n_fft=2560,hop_length=32):
    #fast_fourier_transf = np.fft.fft(waveform)
   
    audio_stft = librosa.feature.melspectrogram(y=waveform,n_fft=n_fft,hop_length=hop_length,fmin = 20,fmax = sample//2)
    log_spectro = librosa.power_to_db(audio_stft)
    # librosa.display.specshow(log_spectro, sr=sample, x_axis='time', y_axis='mel',cmap = 'magma')
    
    
    return log_spectro

In [147]:
def mel_freq(waveform,sample):
    mfccs = librosa.feature.mfcc(waveform, sr=sample,dct_type = 2)
    data=np.mean(librosa.feature.mfcc(waveform,sr=sample,n_mfcc=100).T,axis=0)
    return data

In [148]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]

In [149]:
def create_feature(dataset,num,train=False):

    for i, data in enumerate(dataset):
        sample_rate = data[1]
        waveform = data[0]
        sample_rate_cp = data[1]
        waveform_cp = data[0]  
        # mel_spectrogram_spectrogram = mel_spectrogram(waveform,sample_rate)
        mel_freq_spectrogram = mel_freq(waveform_cp,sample_rate_cp)
        if train == False:
            x_test.append(mel_freq_spectrogram)
            y_test.append(num)
        else:
            x_train.append(mel_freq_spectrogram)
            y_train.append(num)

In [150]:
create_feature(burping_train_data,num=0,train=True)
create_feature(poop_train_data,num=1,train=True)
create_feature(hungry_train_data,num=2,train=True)
create_feature(tired_train_data,num=3,train=True)
create_feature(discomfort_train_data,num=4,train=True)

c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[ 0.00177924 -0.00799471 -0.01225367 ... -0.00759041  0.00428651
  0.00536661] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[ 0.00177924 -0.00799471 -0.01225367 ... -0.00759041  0.00428651
  0.00536661] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[-0.00318202 -0.00577083 -0.00303842 ...  0.0100263   0.00637814
  0.00114381] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[-0.00318202 -0.00577083 -0.

In [151]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [152]:
def dummy(data,numarr=5):
    new_data =[]
    for i in data:
        lab_data = []
        for j in range(0,numarr):
            lab_data.append(0)
        #lab_data =[0,0]
        lab_data[i] = 1
        new_data.append(lab_data)
    return new_data

In [153]:
# model=tf.keras.Sequential()
# model.add(tf.keras.layers.Dense(input_shape=(100,), units=100 ,activation='relu'))
# # model.add(tf.keras.layers.Embedding(input_dim=100, output_dim=64))
# model.add(tf.keras.layers.Dense(128,activation='relu'))
# # model.add(tf.keras.layers.SimpleRNN(128))
# # model.add(tf.keras.layers.LSTM(200,activation='relu'))
# model.add(tf.keras.layers.Dense(64,activation='relu'))
# model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [154]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(units=64, input_shape=(None, 100)),
    tf.keras.layers.SimpleRNN(units=128),
    tf.keras.layers.Dense(units=5, activation='softmax')
])

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: Input 0 of layer "simple_rnn_8" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

In [ ]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 100)               10100     
                                                                 
 dense_32 (Dense)            (None, 128)               12928     
                                                                 
 dense_33 (Dense)            (None, 64)                8256      
                                                                 
 dense_34 (Dense)            (None, 5)                 325       
                                                                 
Total params: 31,609
Trainable params: 31,609
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adamax(1e-4)# modify weight in Adam
metrics = ['accuracy']
model.compile (optimizer = optimizer,loss =  loss_fn,metrics = metrics)

In [ ]:
def create_weight(n_sample,n_class,n_class_sample):
    weight = n_sample/(n_class*n_class_sample)
    return weight 

In [ ]:

def createdict(num):
    lab_data = []
    for i in range(0,num):
        lab_data.append(i)
    dic = dict.fromkeys(lab_data, 0)
    return dic

In [ ]:
def createWeight(class_count,num_classes):
    class_weights = createdict(num_classes)
    for i in range(num_classes):
        class_weights[i]=create_weight(np.sum(class_count),num_classes,class_count[i])
    return class_weights

In [ ]:
class_count=[]

In [ ]:
class_count

[]

In [ ]:
for i in range(0,5):
    class_count.append(np.count_nonzero(y_train == i))
class_count = np.array(class_count)
class_weights = createWeight(class_count,5)
   

In [ ]:
class_weights

{0: 0.7659574468085106,
 1: 1.0,
 2: 1.1612903225806452,
 3: 0.9,
 4: 1.3846153846153846}

In [ ]:
y_train = np.array(dummy(y_train))

In [ ]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
history = model.fit(x_train, y_train, epochs=100,validation_split=0.2,class_weight = class_weights)

Epoch 1/100


c:\Users\acer\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


5/5 [==============================] - 0s 28ms/step - loss: 14.6641 - accuracy: 0.3333 - val_loss: 6.2417 - val_accuracy: 0.0278
Epoch 2/100
5/5 [==============================] - 0s 6ms/step - loss: 10.8600 - accuracy: 0.2847 - val_loss: 5.8519 - val_accuracy: 0.1111
Epoch 3/100
5/5 [==============================] - 0s 7ms/step - loss: 8.8705 - accuracy: 0.3125 - val_loss: 6.1430 - val_accuracy: 0.1667
Epoch 4/100
5/5 [==============================] - 0s 6ms/step - loss: 7.9236 - accuracy: 0.3264 - val_loss: 6.7066 - val_accuracy: 0.0556
Epoch 5/100
5/5 [==============================] - 0s 8ms/step - loss: 7.2175 - accuracy: 0.3472 - val_loss: 6.6832 - val_accuracy: 0.0556
Epoch 6/100
5/5 [==============================] - 0s 8ms/step - loss: 6.3914 - accuracy: 0.3611 - val_loss: 6.3687 - val_accuracy: 0.1667
Epoch 7/100
5/5 [==============================] - 0s 8ms/step - loss: 5.5751 - accuracy: 0.3681 - val_loss: 6.1412 - val_accuracy: 0.1667
Epoch 8/100
5/5 [===================